In [57]:
import glob
import pandas as pd
import os
import json

In [62]:
inputdir = "websitedatatemp"
outputdir = "cleanedwebsitedata"

for tt in ["user", "spider"]:
    for at in ["desktop", "mobile-dev", "mobile-app"]:
        for domain in ["en.wikipedia", "de.wikipedia", "fr.wikipedia", "es.wikipedia", "ru.wikipedia", "zh.wikipedia"]:
            for year in [2019]:
                for month in [9]:
                    for day in range(1,32):

                        jsondic = {i:[] for i in range(24)}
                        filepathpattern = f"./{inputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}-*.csv"
                        filepaths = glob.glob(filepathpattern)
                        
                        if not filepaths:
                            continue
                            print(f"{filepathpattern} IS EMPTY")
                        # elif len(filepaths) != 24:
                        #     continue
                            # print(f"{filepathpattern} DOES NOT CONTAIN 24 HOURS")
                        
                        for filepath in filepaths:
                            hour = int(filepath.split("-")[-1].split(".")[0])
                            with open(filepath, "r") as f:
                                for line in f.read().split("\n"):
                                    x, y = line.split(",")
                                    jsondic[hour].append({"x":int(x), "y":int(y)})

                        outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}.json"
                        outfiledir = "/".join(outfilepath.split("/")[:-1])

                        if not os.path.exists(outfiledir):
                            print()
                            os.makedirs(outfiledir)

                        with open(outfilepath, "w") as f:
                            json.dump(jsondic, f)

In [60]:
str({"a"})

"{'a'}"

In [33]:
f.read()

ValueError: I/O operation on closed file.

In [24]:
df

'{"0":{"0":1234,"1":123,"2":12,"3":2,"4":1},"1":{"0":4,"1":5,"2":6,"3":10,"4":10000}}'

In [15]:
f"./{inputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}-*.csv"

'./websitedatatemp/spider/mobile-app/zh.wikipedia/2019-09-31-*.csv'

In [9]:
"/".join(["1", "2", "3"])

'1/2/3'

In [6]:
'./websitedatatemp/spider'.split("/")

['.', 'websitedatatemp', 'spider']